In [ ]:
import pennylane as qml
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import pandas as pd

In [ ]:
def abs_error(m1, m2):
    return abs(m1 - m2)


def relative_error(m1, m2):
    return abs_error(m1, m2) / abs(m1)


def mse(m1, m2, variance) -> float:
    return math.sqrt(abs(m1 - m2) ** 2 + variance)


def print_data(name, mean, true_mean, samples__, variance):
    return {
        "Name": name,
        "Mean": mean,
        "Absolute Error": abs_error(true_mean, mean),
        "Relative Error": relative_error(true_mean, mean),
        "MSE": mse(true_mean, mean, variance),
        "Variance": variance,
        "Samples": samples__,
    }

In [ ]:
m = 5
M = 2**m

xmax = np.pi
xs = np.linspace(-xmax, xmax, M)

probs = np.array([norm().pdf(x) for x in xs])
probs /= np.sum(probs)

In [ ]:
samples = []
for i in range(1, 7):
    samples_qmc = 2**i
    error = 1 / samples_qmc
    samples_mc = round(1 / error**2)

    samples.append(
        {
            "Error": error,
            "Classical Samples": samples_mc,
            "Quantum Samples": samples_qmc,
        }
    )


df_samples = pd.DataFrame(samples)
quantum_samples = [int(math.log2(n)) for n in df_samples["Quantum Samples"].to_list()]
classical_samples = df_samples["Classical Samples"].to_list()

print(df_samples)
print(df_samples.to_latex(index=False, float_format="%.4f"))

In [ ]:
def func_f(x):
    return np.sin(x) ** 2


true_mean = math.sinh(1) / math.exp(1)


def mc_mean(numSamples):
    sampleData = []
    for _ in range(numSamples):
        sampleData.append(np.random.choice(xs, p=probs))
    values: list = func_f(sampleData)
    MCMean: float = np.mean(values)
    return MCMean, numSamples

In [ ]:
mse_values = []
abs_values = []
num_samples_cmc = []
data_cmc = []
variances = []
abs_values = []

for samples_ in classical_samples:
    num_samples_cmc.append(samples_)
    answer, num = mc_mean(samples_)

    results = []
    for _ in range(5):
        result, numSamples = mc_mean(samples_)
        results.append(result)

    variance = np.var(results)

    answer = print_data(
        "MC Mean",
        mean=answer,
        true_mean=true_mean,
        samples__=numSamples,
        variance=variance,
    )

    data_cmc.append(answer)
    mse_values.append(answer.get("MSE"))

In [ ]:
df_cmc = pd.DataFrame(data_cmc)
print(df_cmc)
df_cmc = df_cmc.drop("Name", axis=1)
print(
    df_cmc.to_latex(
        index=False,
    )
)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(num_samples_cmc, mse_values, marker="o", linestyle="-", color="b")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Number of Samples (log scale)")
plt.ylabel("Mean Square Error (log scale)")
plt.title("Mean Square Error vs Number of Samples for Classical Monte Carlo")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.show()

In [ ]:
# TODO Quantum Amplitude Estimation

In [ ]:
def func(i):
    return np.sin(xs[i]) ** 2


mse_values_qc = []
num_samples = []
data = []
for n in quantum_samples:
    num_samples.append(2**n)
    N = 2**n

    target_wires = range(m + 1)
    estimation_wires = range(m + 1, n + m + 1)

    dev = qml.device("default.qubit", wires=(n + m + 1))

    @qml.qnode(dev)
    def circuit():
        qml.templates.QuantumMonteCarlo(
            probs,
            func,
            target_wires=target_wires,
            estimation_wires=estimation_wires,
        )
        return qml.probs(estimation_wires)

    results = []
    for _ in range(4, 17):
        qmc_probs = circuit()
        phase_estimated = np.argmax(circuit()[: int(N / 2)]) / N
        answer = (1 - np.cos(np.pi * phase_estimated)) / 2
        results.append(answer)
    variance = np.var(results)
    answer = np.mean(results)

    format_data = print_data("QMC", answer, true_mean, 2**n, variance=variance)

    mse_values_qc.append(format_data.get("MSE"))
    data.append(format_data)

In [ ]:
fig, ax = qml.draw_mpl(circuit, level="device", style="black_white")()
fig.savefig(f"../circuits/pl_qmc.png", dpi=330, bbox_inches="tight")

In [ ]:
df = pd.DataFrame(data)
print(df)
df = df.drop("Name", axis=1)
print(
    df.to_latex(
        index=False,
    )
)

In [ ]:
abs_error_list = df["Absolute Error"].to_list()
samples_numbers = df["Samples"].to_list()


plt.plot(samples_numbers, mse_values_qc, marker="o", linestyle="-", color="b")
plt.xlabel("Number of Samples", fontsize=16)
plt.ylabel("Absolute Error", fontsize=16)
plt.xscale("log", base=10)
plt.yscale("log")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.title("Mean Square Error vs Number of Samples for Quantum Monte Carlo")

In [ ]:
samples_numbers = df["Samples"].to_list()

plt.plot(
    samples_numbers,
    mse_values_qc,
    marker="o",
    linestyle="-",
    color="b",
    label="Quantum Monte Carlo",
)
plt.plot(
    num_samples_cmc,
    mse_values,
    marker="o",
    linestyle="-",
    color="g",
    label="Classical Monte Carlo",
)
plt.legend()
plt.xlabel("Number of Samples", fontsize=16)
plt.ylabel("Mean Square Error", fontsize=16)
plt.xscale("log", base=10)
plt.yscale("log")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.title("Mean Square Error vs Number of Samples of CMC and QMC")